In [1]:
import pdbufr
import sys
import traceback
 
from math import isnan

import plotly.express as px
from shapely.geometry import LineString

from ipywidgets import interact
import os
import birdy
import geopandas as gpd
import pandas as pd
from datetime import datetime, timedelta
import ipyleaflet
import ipywidgets as widgets

from Magics import macro as magics
from IPython.display import display
from Magics.macro import *

import numpy as np
import xarray as xr

import warnings
warnings.filterwarnings("ignore")

In [2]:
active_storms_nc = xr.open_dataset('data/IBTrACS.ACTIVE.v04r00.nc')
active_storms_csv = pd.read_csv('data/ibtracs.ACTIVE.list.v04r00.csv', header=[0,1])
storms = active_storms_csv['NAME'].squeeze().unique().tolist()
df_storm1 = active_storms_csv[active_storms_csv.NAME.squeeze() == storms[0]]

In [3]:
## FUNCTION TO PLOT THE PAST TRACK OF A CYCLONE ## 

def plot_observed_track(storm_name, track_colour, lat_boundaries, lon_boundaries):
    toplot = []
    
    # storm data preparation for plotting
    df_active_storms = pd.read_csv('data/ibtracs.ACTIVE.list.v04r00.csv', header=[0,1])
    df_storm = df_active_storms[df_active_storms.NAME.squeeze() == storm_name]
    lat = df_storm.LAT.squeeze().to_numpy(dtype='float')
    lon = df_storm.LON.squeeze().to_numpy(dtype='float')
    start_str = df_storm.ISO_TIME.squeeze().iloc[0]
    end_str = df_storm.ISO_TIME.squeeze().iloc[len(df_storm)-1]
    start = datetime.strptime(start_str, '%Y-%m-%d %H:%M:%S')
    end = datetime.strptime(end_str, '%Y-%m-%d %H:%M:%S')
    
    # settings of the geographical area
    bottom, up = lat_boundaries
    left, right = lon_boundaries
    area = mmap(
        subpage_map_projection="cylindrical",
        subpage_lower_left_longitude=int(left),
        subpage_lower_left_latitude=int(bottom),
        subpage_upper_right_longitude=int(right),
        subpage_upper_right_latitude=int(up),
    )
    toplot.append(area)
    
    # settings of the coastline
    coast = mcoast(
        map_coastline_land_shade = "on",
        map_coastline_land_shade_colour = "cream",
        map_coastline_sea_shade = "on",
        map_coastline_sea_shade_colour = "#70CEE2",
        # map_cities = "on",
        map_grid_line_style = "dash",
        map_grid_colour = "black",
        map_label = "on",
        map_label_colour = "brown",
        map_coastline_colour = "brown",
    )
    toplot.append(coast)

    data = minput(
        input_type = 'geographical',
        input_x_values = lon,
        input_y_values = lat,
    )

    line = msymb(
        symbol_type='marker',
        symbol_marker_index = 28,
        symbol_colour = track_colour,
        symbol_height = 0.20,
        symbol_text_font_colour = "black",       
        symbol_connect_line ='on'
    )

    toplot.append(data)
    toplot.append(line)
    
    title = mtext(
        text_lines= [f"<font colour='navy'> <b> Cyclone {storm_name} observed trajectory </b> </font>",
                    f"{start.strftime('%d %b %Y %H:%M')} - {end.strftime('%d %b %Y %H:%M')}"],
        text_justification= 'centre',
        text_font_size= 0.7,
        # text_font_style= 'bold',
        text_mode='title',
    )
    toplot.append(title)
    
    display(plot(toplot))

In [4]:
## DEFINE WIDGETS FOR INTERACTIVE MAP ##

# Cyclone dropdown selection
storms = active_storms_csv['NAME'].squeeze().unique().tolist()
cyclone = widgets.Dropdown(
    options = storms,
    description = 'Active Storms:',
    disabled=False,
)
cyclone.style.description_width = '90px'

# Longitude slider
longitude = np.arange(-180,185,5)
longitude_slider = widgets.SelectionRangeSlider(
    options=longitude,
    index=(0, len(longitude)-1),
    description='Longitude:',
    orientation='horizontal',
    layout={'width': '400px'},
    disabled=False,
)

longitude_slider.style.description_width = '68px'
longitude_slider.style.handle_color = 'lightgreen'

# Latitude slider
latitude = np.arange(-90,95,5)
latitude_slider = widgets.SelectionRangeSlider(
    options=latitude,
    index=(0, len(latitude)-1),
    description='Latitude:',
    orientation='horizontal',
    layout={'width': '400px'},
    disabled=False,
)

latitude_slider.style.description_width = '58px'
latitude_slider.style.handle_color = 'lightgreen'

# Create widget for track colour in the map
colour = widgets.Dropdown(
    options = ["red", "blue", "green", "yellow", "purple", "orange", "cyan", "black"],
    value = "red",
    description = "Track Colour:",
    disabled = False,
)

colour.style.description_width = '82px'

In [5]:
widgets.interactive(plot_observed_track, storm_name=cyclone, track_colour=colour, lat_boundaries=latitude_slider, lon_boundaries=longitude_slider)

interactive(children=(Dropdown(description='Active Storms:', options=('KHANUN', 'DORA', 'LAN', 'FERNANDA', 'GR…